# The Elite Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [1]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [2]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [50]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    """ pi: list with probability values
        return roulette proportions"""
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    """ ppi: list of roulette proportions
        Pop: list of corresponding item   [item1, ... , itemN] """
    
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost and Rank Ants

In [65]:
# calculate cost of each route
def Cost(Pop, graph):
    """ Pop: List of All Items in the e.g [[item1], ... , [itemN]]
    graph: cost matrix or graph 
    returns a cost vector for corresponding item """
    
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += graph[route[j]][route[j+1]]
        cost += graph[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Sort Chromosomes from Best to Worst
def Rank(Pop, graph):
    """ returns sorted Ants """
    cost = Cost(Pop, graph)
    ans = [ x for _,x in sorted(zip(cost, Pop))]
    #cost = [ _ for _,x in sorted(zip(cost, Pop))]  # need to use the true cost
    return ans

# Initialize Pheromone 

In [83]:
# AS initialization
def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# EAS initialization
def EASTSPher(graph, e, m, rho):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    numer = e + m
    den = rho*cost
    pher = [[numer/den for i in range(n)] for j in range(n)]
    
    return pher


# EAS initialization
def MMTSPher(graph, tmax, rho = 0.1):
    # small evaporation 
    
    n = len(graph)
    pher = [[(1-rho)*tmax for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [84]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        
    return connections

# Find Path 

In [85]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Get Elite Route and Connections

In [86]:
def Elite(Elite, ant, graph):
    
    if Cost([ant], graph)[0] < Cost([Elite], graph)[0]:
        return ant, Cost([ant], graph)[0]
    else:
        return Elite, 0

def connections(L):
    """L is a list
    one way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
    
    cons.append((L[-1], L[0]))
    
    return cons

def connectionsE(L):
    """L is a list
    Two way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
        cons.append( (L[i], L[i-1]) )
    
    cons.append((L[-1], L[0]))
    cons.append((L[0], L[1]))
    
    return cons

# Update Pheromone 

In [207]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Elite Ant System
def RBASUpPheromone(route, matrix, w, r):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        matrix[route[i]][route[i+1]] += (w-r)*tau 
        matrix[route[i+1]][route[i]] += (w-r)*tau 
        
    matrix[route[-1]][route[0]] += (w-r)*tau
    matrix[route[0]][route[-1]] += (w-r)*tau

    return matrix

# Ranked based Ant System
def ElitePheromone(Elite, matrix, w):
    """route : route taken
    matrix: pheromone matrix"""
    
    bstau = 1/Cost([Elite], matrix)[0]
    #tau = 1
    
    for i in range(len(Elite)-1):
        matrix[Elite[i]][Elite[i+1]] += (w*bstau) 
        matrix[Elite[i+1]][Elite[i]] += (w*bstau) 
        
    matrix[Elite[-1]][Elite[0]] += (w*bstau)
    matrix[Elite[0]][Elite[-1]] += (w*bstau)

    return matrix

# Min-Max Ant System
def MMPheromone(Elite, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    T = Cost([Elite], matrix)[0]
    bstau = 1/T
    #tau = 1
    
    for i in range(len(Elite)-1):
        matrix[Elite[i]][Elite[i+1]] += bstau 
        matrix[Elite[i+1]][Elite[i]] += bstau 
        
    matrix[Elite[-1]][Elite[0]] += bstau
    matrix[Elite[0]][Elite[-1]] += bstau

    return matrix

# Rank Based Ant System
def MMUpPheromone(route, EliteRoute, matrix):  # problematic when multiplied by (w-r)
    """route : route taken
    EliteRoute: Best Route found thus far
    matrix: pheromone matrix
    e: weighting factor for elite pheromone"""
    
    Tbs = connectionsE(EliteRoute)
    
    bstau = 1/Cost([EliteRoute], matrix)[0]
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        if (route[i], route[i+1]) in Tbs:
            matrix[route[i]][route[i+1]] += tau 
            matrix[route[i+1]][route[i]] += tau 
    
    # add pheromone to last and first route for TSP
    if (route[-1], route[0]) in Tbs:
        matrix[route[-1]][route[0]] += tau
        matrix[route[0]][route[-1]] += tau

    return matrix

# Evaporate Pheromone 

In [208]:
def EvPheromone(matrix, rho, tmax, tmin):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = tmin
    return matrix

def EvPheromoneNeg(matrix, rho, tmax, tmin):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
            
            if matrix[i][j] > tmax:
                matrix[i][j] = tmax
            
            if matrix[i][j] < tmin:
                matrix[i][j] = tmin
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [209]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [210]:
def AS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, a = 10, MaxStag = 10, runs = 50, Pher = None):
    
    
    GOAT = [i for i in range(len(Ngraph))]  # initial best ant
    tmax = 1/(rho*(Cost([GOAT], Ngraph)[0]))
    tmin = tmax/a
    
    # to compare if the best-so-far has changed
    PGOAT = GOAT.copy()
    
    # Initialize Pheromone
    if Pher == None:
        #Pher = TSPher(Ngraph)
        Pher = MMTSPher(Ngraph, tmax)
        
    # Stagnation Count   MaxStag is maximum attenable stag to initiate reinitalization
    stag = 0
        
    starthob = random.choice([i for i in range(len(Ngraph))])

    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
            GOAT, changed = Elite(GOAT, stof, Ngraph)
            
        #readjust pheromone limits
        if GOAT != PGOAT:
            tmax =  1/(rho*(Cost([GOAT], Ngraph)[0]))
            tmin = tmax/a
            stag = 0
        
        # update stagnation count and reinitialize pheromone if necessary
        if GOAT == PGOAT:
            stag += 1
            if stag == MaxStag:
                Pher = MMTSPher(Ngraph, tmax)
                print(i, " Stagnation Found at :", stag , "  Reinitialization Innitaited")
                stag = 0
        
        PGOAT = GOAT.copy()
        # Rank Ants by sorting and keep top w-1
        Ranked = Rank(Ants, Ngraph)[0]
        B = random.choice([Ranked, GOAT])

        #Pheromone update best ant 
        #for r,j in enumerate([B]):
            #r += 1
            #MMPheromone(j, Pher)
          
        #Pheromone update all Ants
        for r,j in enumerate(Ants):
            r += 1
            #MMUpPheromone(j, GOAT, Pher)
            #MMUpPheromone(j, Ranked, Pher)
            MMUpPheromone(j, B, Pher)
        
        #ElitePheromone(GOAT, Pher, w)
        
        #Pheromone evaporation
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho, tmax, tmin)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            E = Cost([GOAT], Ngraph)[0]
            print("Evolution ", i, " Elite Cost = ", E,  " Minimum Cost = ", min(C), "  and Maximum Cost = ", max(C))
            #print("<<<<<<<<  The Best Route IS  >>>>>>>> ", GOAT)
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [211]:
# Toy Network graph 
D = 10
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

np.array(graph)

array([[ 0,  2,  9,  9,  7,  8,  5, 10,  6,  9],
       [ 2,  0,  5,  4,  5,  6,  8,  8, 10,  6],
       [ 9,  5,  0, 10,  2,  9,  2,  3,  4, 10],
       [ 9,  4, 10,  0,  4,  6,  4, 10,  7,  3],
       [ 7,  5,  2,  4,  0,  4,  6,  5,  3,  9],
       [ 8,  6,  9,  6,  4,  0,  5,  5,  9,  4],
       [ 5,  8,  2,  4,  6,  5,  0,  2,  5,  4],
       [10,  8,  3, 10,  5,  5,  2,  0,  8,  7],
       [ 6, 10,  4,  7,  3,  9,  5,  8,  0,  8],
       [ 9,  6, 10,  3,  9,  4,  4,  7,  8,  0]])

In [212]:
# Find Shortest Path with the Algorithm

ants = len(graph)
alpha = 1
beta = 3
rho = 0.5
count = len(graph)*3
a = 10
runs = 200
count = int(runs/4)

AS(graph, ants, alpha, beta, rho, a, count, runs)

Evolution  0  Elite Cost =  37  Minimum Cost =  37   and Maximum Cost =  54
Evolution  10  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  46
Evolution  20  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  46
Evolution  30  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  44
Evolution  40  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  44
Evolution  50  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  43
54  Stagnation Found at : 50   Reinitialization Innitaited
Evolution  60  Elite Cost =  33  Minimum Cost =  36   and Maximum Cost =  47
Evolution  70  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  46
Evolution  80  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  50
Evolution  90  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  45
Evolution  100  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  48
104  Stagnation Found at : 50   Reinitialization Innitaited
Evolution  110  Elite Cost =  33  

(array([[0, 1, 3, 9, 5, 7, 6, 2, 4, 8],
        [0, 1, 2, 4, 8, 9, 3, 5, 7, 6],
        [0, 1, 3, 9, 5, 7, 6, 2, 4, 8],
        [0, 1, 3, 9, 5, 6, 7, 2, 4, 8],
        [0, 1, 3, 9, 5, 7, 6, 2, 4, 8],
        [0, 1, 3, 9, 5, 6, 2, 4, 8, 7],
        [0, 1, 9, 3, 6, 7, 2, 4, 8, 5],
        [0, 1, 3, 9, 8, 4, 2, 6, 7, 5],
        [0, 1, 3, 9, 5, 7, 6, 2, 4, 8],
        [0, 1, 3, 9, 5, 7, 6, 2, 4, 8]]), 33)

# Soving Real TSP problems 

In [227]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 3
rho = 0.5
a = len(graph)
runs = 1000
counts = int(runs/5)

AS(graph, ants, alpha, beta, rho, a, counts, runs)

Evolution  0  Elite Cost =  45441.87095262647  Minimum Cost =  45441.87095262647   and Maximum Cost =  67838.2240109103
Evolution  50  Elite Cost =  35091.65687879188  Minimum Cost =  35091.65687879188   and Maximum Cost =  50773.50495268263
Evolution  100  Elite Cost =  34917.97325484356  Minimum Cost =  34917.97325484356   and Maximum Cost =  51127.534573258534
Evolution  150  Elite Cost =  34873.92392660112  Minimum Cost =  34873.92392660112   and Maximum Cost =  54851.53045110976
Evolution  200  Elite Cost =  34424.657156907844  Minimum Cost =  34424.657156907844   and Maximum Cost =  51589.74556700699
Evolution  250  Elite Cost =  33994.20866319169  Minimum Cost =  33994.20866319169   and Maximum Cost =  47748.133042230824
Evolution  300  Elite Cost =  33755.35204159012  Minimum Cost =  33755.35204159012   and Maximum Cost =  49491.06111965398
Evolution  350  Elite Cost =  33755.35204159012  Minimum Cost =  33755.35204159012   and Maximum Cost =  48249.471351307024
Evolution  400 

(array([[31, 38, 33, ..., 42, 29,  5],
        [31, 38, 20, ...,  9, 23, 16],
        [31, 38, 22, ...,  2, 33, 13],
        ...,
        [31, 38, 20, ..., 36,  5, 27],
        [31, 38, 20, ...,  3, 25,  5],
        [31, 38, 20, ..., 36, 18, 26]]), 33701.26176127258)

In [228]:
# burma14

import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 3
rho = 0.5
a = len(graph)
runs = 1000
counts = int(runs/5)

AS(graph, ants, alpha, beta, rho, a, counts, runs)

Evolution  0  Elite Cost =  38.13241680045596  Minimum Cost =  38.13241680045596   and Maximum Cost =  50.1560589603592
Evolution  50  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  47.87538670983891
Evolution  100  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  44.932290905537236
Evolution  150  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  47.000151309857664
Evolution  200  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  43.597817651709114
Evolution  250  Elite Cost =  30.878503892588  Minimum Cost =  30.878503892588   and Maximum Cost =  45.964013012637935
Evolution  300  Elite Cost =  30.878503892588  Minimum Cost =  30.878503892588   and Maximum Cost =  44.71571627340581
Evolution  350  Elite Cost =  30.878503892588  Minimum Cost =  30.878503892588   and Maximum Cost =  42.66104646312548
Evolution  400  Elit

(array([[ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2,  3],
        [ 4,  5, 11,  3,  2, 13,  1,  0,  9,  8, 10,  7, 12,  6],
        [ 4,  5, 11,  6, 12,  7,  0,  1, 13,  2,  3, 10,  8,  9],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  0,  1, 13,  2,  3,  9],
        [ 4,  3,  2, 13,  1,  0,  9,  8, 10,  7, 12,  6, 11,  5],
        [ 4,  3,  2, 13,  1,  0,  9,  8, 10,  7, 12,  6, 11,  5],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  3,  2],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  3,  2],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2,  3],
        [ 4,  6, 12,  7, 10,  8,  9,  0,  1, 13,  2, 11,  5,  3],
        [ 4,  3, 11,  5, 13,  2,  7, 12,  6,  8, 10,  9,  0,  1]]),
 30.878503892588)